In [4]:
import pandas as pd
from unidecode import unidecode

# Função de limpeza APRIMORADA: Deixa maiúsculo, remove acentos E remove espaços no início/fim
def normalizar_nome(nome):
    return unidecode(str(nome)).upper().strip()

# --- PARTE 1: PROCESSAR DADOS DO CAGED ---
print("Processando dados do CAGED...")
caminho_caged = 'C:/Users/felps/mec-plataforma-mapeamento-demanda-ept/data/3-tabelas_Agosto de 2025.xlsx'
df_caged_raw = pd.read_excel(caminho_caged, sheet_name='Tabela 3', header=4)
df_caged_raw.rename(columns={'Unnamed: 6': 'Saldo'}, inplace=True)

# Adicionamos .copy() aqui para evitar o aviso "SettingWithCopyWarning"
df_caged_mg = df_caged_raw[df_caged_raw['UF'] == 'MG'].copy()

df_caged_mg['Municipio_Nome'] = df_caged_mg['Município'].str.replace('Mg-', '', regex=False)
top_10_caged = df_caged_mg.sort_values(by='Saldo', ascending=False).head(10)
top_10_caged = top_10_caged[['Municipio_Nome', 'Saldo']]

# Aplicando a nova Super-Limpeza
top_10_caged['chave_municipio'] = top_10_caged['Municipio_Nome'].apply(normalizar_nome)
print("Top 10 municípios do CAGED processados e normalizados.\n")


# --- PARTE 2: PROCESSAR DADOS DO IBGE ---
print("Processando dados do IBGE...")
caminho_ibge = 'C:/Users/felps/mec-plataforma-mapeamento-demanda-ept/data/Agregados_por_municipios_alfabetizacao_BR.csv'
df_ibge_raw = pd.read_csv(caminho_ibge, sep=';', encoding='latin-1')
df_ibge_mg = df_ibge_raw[df_ibge_raw['CD_MUN'].astype(str).str.startswith('31')].copy()
colunas_uteis = {'NM_MUN': 'Municipio_Nome', 'V00650': 'Populacao_15_a_19'}
df_ibge_final = df_ibge_mg[list(colunas_uteis.keys())].rename(columns=colunas_uteis)
df_ibge_final['Populacao_15_a_19'] = pd.to_numeric(df_ibge_final['Populacao_15_a_19'], errors='coerce')
df_ibge_final.dropna(inplace=True)
df_ibge_final['Populacao_15_a_19'] = df_ibge_final['Populacao_15_a_19'].astype(int)

# Aplicando a nova Super-Limpeza
df_ibge_final['chave_municipio'] = df_ibge_final['Municipio_Nome'].apply(normalizar_nome)
print("Dados de população do IBGE processados e normalizados.\n")


# --- PARTE 3: CRUZAMENTO FINAL ---
print("Cruzando os dados usando a chave super-limpa...")
df_final = pd.merge(
    top_10_caged, 
    df_ibge_final,
    on='chave_municipio',
    how='left'
)

# Calculamos a nova métrica
df_final['Vagas_por_1000_jovens'] = (df_final['Saldo'] / df_final['Populacao_15_a_19']) * 1000


# --- EXIBIÇÃO DO RESULTADO ---
print("\n" + "="*70)
print("     RESULTADO FINAL: VAGAS GERADAS VS. PÚBLICO ALVO (100% CORRIGIDO)")
print("="*70)

colunas_finais = ['Municipio_Nome_x', 'Saldo', 'Populacao_15_a_19', 'Vagas_por_1000_jovens']
print(df_final[colunas_finais].rename(columns={'Municipio_Nome_x': 'Município'}))

# --- PARTE FINAL: SALVAR O RESULTADO ---
# Salvamos a tabela final em um arquivo CSV dentro da pasta 'data'
caminho_resultado = 'C:/Users/felps/mec-plataforma-mapeamento-demanda-ept/data/resultado_final_caged_ibge.csv'

df_final.to_csv(caminho_resultado, index=False, sep=';', decimal=',')

print(f"\nResultado final salvo com sucesso em: {caminho_resultado}")

Processando dados do CAGED...
Top 10 municípios do CAGED processados e normalizados.

Processando dados do IBGE...
Dados de população do IBGE processados e normalizados.

Cruzando os dados usando a chave super-limpa...

     RESULTADO FINAL: VAGAS GERADAS VS. PÚBLICO ALVO (100% CORRIGIDO)
                   Município Saldo  Populacao_15_a_19 Vagas_por_1000_jovens
0             Belo Horizonte  1479             160573              9.210764
1                      Betim   991              28696              34.53443
2              Montes Claros   291              27465             10.595303
3                    Itabira   279               7891             35.356736
4                 Ouro Preto   269               5132             52.416212
5               Juiz de Fora   257              35028              7.336988
6                  Nova Lima   239               8421             28.381427
7                 Leopoldina   221               3411             64.790384
8              Lagoa Formo

C:\Users\Felps\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
